In [26]:
!pip install optuna==3.3.0
!pip install scikit-learn 
!pip install xgboost==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 42.2 MB/s eta 0:00:00
  Attempting uninstall: optuna
    Found existing installation: optuna 4.0.0
    Uninstalling optuna-4.0.0:
      Successfully uninstalled optuna-4.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 4.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.1
    Uninstalling xgboost-2.1.1:
      Successfully uninstalled xgboost-2.1.1


In [27]:
from datetime import datetime
import json
import sys
import os
from xgboost_class import XGBoostOptimizer
import hopsworks

In [28]:
proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()


Python Engine initialized.


Connection closed.

Logged in to project, explore it here https://hopsworks0.logicalclocks.com/p/119


In [11]:
fg = fs.get_feature_group(name="stock_features", version = 1)

fv = fs.get_or_create_feature_view(name="tickers", version=2,
                                  query=fg.select_all(),
                                  labels=['target']
                                  )

Feature view created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fv/tickers/version/2


In [20]:
X, y = fv.training_data()
X = X.drop(["ticker", "t", "aggregated_text"], axis=1)
X


Provenance cached data - overwriting last accessed/created training dataset from 5 to 6.


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.22s) 


,o,h,l,c,v,vw,n,next_c,target_class,c_class,...,vw_lag_5,n_lag_1,n_lag_2,n_lag_3,n_lag_4,n_lag_5,vw_rolling_mean,vw_rolling_std,n_rolling_mean,n_rolling_std
0,56.347000,56.470699,54.740002,56.053001,474394432.0,55.671600,665715,182.679993,1,0,...,219.400696,686939,767281,396069,461337,1357892,200.449221,122.406074,733903.6,3.811515e+05
1,183.800003,187.559998,182.199997,184.759995,84272928.0,184.494095,980516,413.540009,1,1,...,181.254395,898724,394601,834395,1468448,845538,165.173120,41.328904,888341.2,3.824766e+05
2,221.649994,226.449997,221.649994,226.339996,58287572.0,224.942993,767327,128.199997,1,1,...,125.214897,1719241,410828,451167,1599014,1539247,233.277817,140.104566,1143899.4,6.541634e+05
3,179.929993,182.240005,179.490005,182.149994,29260372.0,181.173599,401018,430.320007,1,1,...,178.038193,992088,301608,333183,492815,783928,216.557263,123.439830,580724.4,2.989241e+05
4,172.779999,178.899994,172.600006,178.500000,42257864.0,176.529907,398515,214.110001,1,1,...,210.336594,594026,314764,1932775,3447787,824159,218.946300,121.083728,1422702.2,1.287792e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,121.769997,122.870003,118.739998,120.910004,218971552.0,120.837402,1805344,187.229996,1,0,...,121.100098,366147,385151,785539,1115208,2798194,220.840338,118.690660,1090047.8,1.003996e+06
885,427.190002,431.054993,424.410004,430.160004,11315176.0,428.775208,301608,106.469002,1,1,...,188.259506,333183,492815,783928,897414,573924,182.735843,6.013080,616252.8,2.260109e+05
886,122.779999,124.690002,122.099998,122.589996,172565376.0,123.216904,1308647,114.250000,0,0,...,122.406502,489886,1425749,551646,316348,1720625,245.222578,121.312435,900850.8,6.284957e+05
887,87.239998,88.819000,86.300003,86.402000,362205888.0,87.219803,711632,175.000000,0,0,...,401.590912,766227,2641180,808166,592772,423985,207.048724,116.457896,1046466.0,9.043996e+05


In [16]:
y

,target
0,1.409988
1,4.779999
2,0.709999
3,0.690002
4,5.479996
...,...
884,0.169998
885,2.020004
886,-4.919998
887,-6.089996


In [31]:
alpha_bounds = [None, (0.85, 0.99), (0.01, 0.15)]  # None is for the main model



for ticker in json.loads('["AAPL", "AMZN", "GOOGL", "MSFT", "NVDA", "TSLA"]'):
    
    data_path = ticker
    number_of_trials = 1000
    cores = 4

    time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')

    for alpha_bound in alpha_bounds:
        if alpha_bound is None:
            model_suffix = 'main'
        elif alpha_bound == (0.85, 0.99):
            model_suffix = 'upper'
        else:
            model_suffix = 'lower'

        optimizer = XGBoostOptimizer(X, y, ticker, trial_no=number_of_trials, n_parallel=cores, timestamp=time_stamp)
        optimizer.optimize()
        #alpha_bound=alpha_bound)

        os.makedirs("xgboost/images", exist_ok=True)
        model_save_path = f'xgboost/xgboost_{data_path.lower()}_{model_suffix}-{time_stamp}.json'
        optimizer.save_best_model(model_save_path)

        image_save_path = f'xgboost/images/xgboost_{data_path.lower()}_{model_suffix}-{time_stamp}.png'
        optimizer.plot_results(image_save_path)


Hyperparameter Optimizing:   0%|          | 0/1000 [00:00<?, ?it/s]FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.









Hyperparameter Optimizing:   1%|          | 9/1000 [00:00<00:12, 82.13it/s]FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.









Hyperparameter Optimizing:   2%|▏         | 18/1000 [00:00<00:13, 70.40it/s]FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



XGBoostError: [17:20:28] /workspace/src/tree/updater_colmaker.cc:110: column sample by node is not yet supported by the exact tree method
Stack trace:
  [bt] (0) /srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x22dbbc) [0x7f1e5242dbbc]
  [bt] (1) /srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x802201) [0x7f1e52a02201]
  [bt] (2) /srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x57f154) [0x7f1e5277f154]
  [bt] (3) /srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x580564) [0x7f1e52780564]
  [bt] (4) /srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x5cae68) [0x7f1e527cae68]
  [bt] (5) /srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x6f) [0x7f1e5

Hyperparameter Optimizing:   2%|▏         | 19/1000 [00:14<00:13, 70.40it/s]

In [ ]:
# Create a sklearn model in the Model Registry
fraud_model = mr.sklearn.create_model(
    "xgboost",
    metrics={'accuracy': accuracy},            # Specify metrics for the model
    input_example=X_test.sample().to_numpy(),  # Provide an input example for the model
    model_schema=model_schema,                 # Specify the model schema
)

# Save the created model in the "lending_model" directory
fraud_model.save('xgboost')